# Running SoS workflows on cluster systems

* **Difficulty level**: intermediate
* **Time need to lean**: 30 minutes or less
* **Key points**:
  * SoS workflows recognizes cluster systems (PBS, Torque, Slurm, LSF, and Sun Grid) and make use of allocated nodes to start workers
  * Option `-j` will be ignored when a workflow is executed under a cluster environment because it will be inferred from cluster environment variables.

## Use multiple workers to execute SoS workflows

SoS uses multiple worker processes to execute steps, substeps, and subworkflows. By default, SoS creates `n/2` workers on a local computer with `n` CPU cores, although it limits the default number of workers to 8 when there are more than 16 cores because those computers are most likely shared by multiple users.

The number of workers used by SoS can be controlled by option `-j`, where `-j 4` creates 4 workers so that you will see 5 sos processes (1 master and 4 worker) when you execute a workflow with command

```
sos run script -j 4
```

It is possible to start workers on multiple remote machines by specifying the name of the machine and number of processes on each of them with an extended version of option `-j`. For example

```
sos run script -j 4 node1:4 node2:4
```

will create 12 workers, 4 on localhost on which the master SoS process resides, 4 on `node1` and 4 on `node2`, where `node1` and `node2` can be name or IP address of machines, or an aliaes defined in SoS configuration files. A limitation of starting workers on remote servers is that the remote servers must share the same file systems as the local host so this approach only works for workstations with auto-mount home directories, or computing nodes of cluster systems.

<div class="bs-callout bs-callout-alert" role="alert">
  <h4>Using workers on remote hosts</h4>
    <p>SoS assumes that <b>all local and remote hosts share the same file systems</b> and propagates environment variables such as <code>$PATH</code> to all remote hosts. This ensures that all worker processes have identical running environments. </p>
    <p>You should create and execute external tasks if you would like to execute code on remote computing environments that do not share file systems with the local host.</p>
</div>

## Running SoS workflows on clusters

It is easy to execute SoS workflows on multiple computing nodes of a cluster system such as PBS, Slurm, and IBM LSF. In this case, multiple nodes can be allocated to execute a single SoS workflow, effectively creating a "mini-cluster" inside the cluster system. The master SoS process will be started on one of the computing nodes and start multiple workers on the head and the rest of the allocated computing nodes.

<p align="center">
  <img src="https://vatlab.github.io/sos-docs/doc/media/cluster_execution.jpg">
</p>

An obvious problem with the use of option `-j` to start workers on multiple computing nodes of a cluster system is that the computing resources (nodes) are allocated dynamically by the scheduler so it is difficult to know in advance which computing nodes will be assigned. Because cluster systems usually specify nodes used by a particular job with environment variables (e.g. `PBS_NODEFILE` for a PBS system, and `LSB_MCPU_HOSTS` for a LSF system), SoS addresses this problem by automatically translate node information from these variables to option `-j` if a supported cluster environment is detected. Therefore, a typical way to execute a large workflow on multiple nodes would be the creation of a wrapper script as follows:

```sh
#!/bin/bash
#PBS -N workflow
#PBS -l nodes=4:ppn=8
#PBS -l walltime=10:0:0
#PBS -l mem=10GB
#PBS -m ae

cd /home/jovyan/projects
sos run my_workflow.sos
```

and submit to the cluster with appropriate command (e.g. `qsub workflow.sh`). Note that this script is used for PBS cluster and the syntax for other cluster systems such as LSF and SLURM will vary. There is no need to specify option `-j` because it will be generated automatically by SoS.

SoS currently supports PBS, Torque, Slurm, IBM LSF, and Sun Grid systems. If your system is not supported, you can specify option `-j` explicitly by parsing the environment varialbe specific to your cluster system. It can be easier, however, to [create a ticket](https://github.com/vatlab/sos/issues) or submit a pull request to enable native support to your system.

## Further reading

* [Running external tasks](tasks.html) for how to define and execute tasks externally, and why `-q` should be set to `none`.